### Assignment 3

##### Deliar Mohammadi, 30072994
##### John Zheng,30125258
##### Xinzhou Li, 30066080



In [90]:
import torch
import torchvision
from torchvision import datasets, transforms, models
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn
import torch_optimizer as optim
from torchsummary import summary

import PIL
from PIL import Image
import numpy as np
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import random
import os
import glob
from tqdm import tqdm

### Building Train and Test Data classes

In [91]:
df_csv = pd.DataFrame(pd.read_csv("./Assignment 3 Dataset/train.csv"))
print(' '.join([str((df_csv['label'] == i).sum()) for i in range(100)]))

train_paths = []
train_labels = []
validation_paths = []
validation_labels = []
label_counts = [0 for _ in range(100)]

for i in range(len(df_csv)):
    l = df_csv['label'].iat[i]
    if label_counts[l] < 1:
        validation_paths.append(df_csv['id'].iat[i])
        validation_labels.append(l)
        label_counts[l] += 1
    else:
        train_paths.append(df_csv['id'].iat[i])
        train_labels.append(l)

38 37 41 43 43 44 41 43 41 46 42 43 40 45 40 34 36 41 43 42 44 42 42 45 41 41 39 41 37 39 36 35 46 45 33 44 35 43 38 42 43 41 45 40 40 47 38 44 45 37 43 37 44 30 45 42 39 40 38 43 35 45 45 44 44 44 41 38 49 43 45 45 41 42 44 43 44 40 44 41 39 40 39 44 46 41 42 46 28 34 42 39 40 43 44 42 45 44 45 43


In [92]:
class Data():
    def __init__(self, root, paths, labels, transform=None, ext='.jpg'): # move transform into training loop
        self.paths = paths
        self.labels = labels
        self.transform = transform
        self.root = root
        # print(self.all_file_paths)
        # print(self.all_labels)
        
    '''
    return length of the dataset
    '''
    def __len__(self):
        return len(self.paths)
    
    '''
    get the img,label tuple corresponding to index i
    '''
    def __getitem__(self, i):
        s = (self.root+'/'+self.paths[i])
        img = Image.open(s)
        label = self.labels[i]
        if self.transform:
            img = self.transform(img)
        return img, label
     
    '''
    get the img,label tuple corresponding to index i
    '''
    def get_no_transform(self, i):
        s = (self.root+'/'+self.paths[i])
        img = Image.open(s)
        label = self.labels[i]
        return img, label
    
    '''
    Selects random start point in the dataset and prints 28 images
    Inputs:
        color-> 1 to print color images, 0 for grayscale images
    '''
    def print_samples(self):
        figure, axes = plt.subplots(3, 7, figsize=(18,10))
        axes = axes.flatten()
        i = random.randint(0, len(self)-29)
        
        for axis in axes:
            x = self.get_no_transform(i)
            axis.imshow(x[0])
            label = x[1]
            axis.set_xlabel(label)
            i+=1
        plt.tight_layout()
        plt.show()
    
    '''
    Calculates average width and height of all images in dataset
    '''
    def calc_average_dimension(self):
        totalw = 0
        totalh = 0
        for i in range(len(self.paths)):
            image = (self.get_no_transform(i))[0]
            w, h = image.size
            totalw += w
            totalh += h
        avgw = totalw//len(self)
        avgh = totalh//len(self)
        return avgw, avgh

## Preparing Training & Validation datasets, dataloaders and transforms

In [93]:
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=(-20, 20)),
    transforms.RandomAdjustSharpness(sharpness_factor=2, p=0.4),
    transforms.ColorJitter(brightness=0.2),
    transforms.ColorJitter(contrast=0.5),
    transforms.ColorJitter(saturation=0.5),
    
    transforms.Resize(size=256),     
    torchvision.transforms.CenterCrop(224), #required for resnet50 input: 224x224
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # normalized with imagenet mean and std deviation
])

In [94]:
validation_transform = transforms.Compose([
    transforms.Resize(size=256),     
    torchvision.transforms.CenterCrop(224),
    torchvision.transforms.ToTensor(),
    torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [95]:
train_dataset = Data("./Assignment 3 Dataset", train_paths, train_labels, train_transform)
validation_dataset = Data("./Assignment 3 Dataset", validation_paths, validation_labels, validation_transform)

In [96]:
train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=True)

#### Sampels of training and test data

In [ ]:
validation_dataset.print_samples()

## Building the Model and Training

In [105]:
class A3ResNetModel():
    def __init__(self, device):
        self.device = device
        
        self.model = models.resnet50(weights=True).to(self.device)
        self.freeze_features()
        
        self.fc = nn.Sequential(
            nn.Linear(2048, 1024),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(1024, 512),
            nn.ReLU(inplace=True),
            nn.Dropout(p=0.3),
            nn.Linear(512, 256),
            nn.ReLU(inplace=True),
            nn.Linear(256, 100)
        )
        
        self.model.fc = self.fc
        self.model.to(self.device)
    
    def freeze_features(self):
        for param in self.model.parameters():
            param.requires_grad = False 
    
    def print_model(self):
        summary(self.model, (3,224,224))

        
    def train_model(self, epochs, train_dataloader, optimizer, loss_function):
        self.model.train()
        for i in tqdm(range(epochs)):
            
            epoch_loss = 0
            num_examples = 0
            correct_examples = 0
            
            for batch_index,(images,labels) in tqdm(enumerate(train_dataloader)):
                # reset optimizer for the current batch
                optimizer.zero_grad() 
                # load images and labels
                images = images.to(device)
                labels = labels.to(device)
                # calculate output and loss from output
                outputs = self.model(images)
                loss = loss_function(outputs,labels)
                # loss.backward() calculates all gradients for all params, optimizer.step() updates params using our optimizer
                loss.backward()
                optimizer.step()
                # calculate information for this epoch
                epoch_loss += loss.item()*len(labels)
                num_examples += len(labels)
                # calculate predictions and accuracy
                _,preds = torch.max(outputs,1) # calcualtae max in each column (second value=1 for column)
                correct_examples += (preds==labels).sum().item()
                
            # AFTER epoch we calculate accuracy
            train_accuracy = (correct_examples/num_examples)*100
            print(f'Epoch {i}: Training Accuracy: {train_accuracy:.4f}%, Training Loss: {epoch_loss:.4f}')

            
    def validate_model(self, validation_dataloader):
        self.model.eval() 
        num_examples = 0
        correct_examples = 0
        
        for batch_index,(images,labels) in tqdm(enumerate(validation_dataloader)):
            images = images.to(device)
            labels = labels.to(device)
            outputs = self.model(images)
            num_examples += len(labels)
            _ , preds = torch.max(outputs,1)
            correct_examples += (preds==labels).sum().item()
            
        validation_accuracy = (correct_examples / num_examples)*100
        print(f'Validation Accuracy: {validation_accuracy:.20f}%')


In [106]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [107]:
resnet50 = A3ResNetModel(device)

In [112]:
loss_function = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50.model.parameters(),lr=0.001)

In [109]:
resnet50.train_model(1, train_dataloader, optimizer, loss_function)

127it [07:58,  3.77s/it]

Epoch 0: Training Accuracy: 0.9913%, Training Loss: 19746.6287


In [111]:
resnet50.validate_model(validation_dataloader)

4it [00:10,  2.54s/it]

Validation Accuracy: 1.00000000000000000000%


### Sources

https://pytorch.org/vision/stable/models.html

https://discuss.pytorch.org/t/what-does-it-mean-to-normalize-images-for-resnet/96160